In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt 

In [2]:
def tab(f):
    try:
        tables=tabula.read_pdf(f,
                                  lattice=True,
                                  pages="all",
                                  silent=True,
                                  multiple_tables=True,
                                  pandas_options={'header':None})
        df = pd.DataFrame()
        df = pd.concat([c for c in tables]).drop_duplicates()
    except: pass
    return df

def process(df):
    df = df.replace(r'^\s*$', np.nan, regex=True)
    try:
        df = df.replace(r'NA', np.nan, regex=True)
    except:pass
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
            df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
        except:
            print("\nSBI Column Headers Missing"); pass
        try:
            df = df.drop(["Init."], axis=1)
        except:
            try:
                df = df.drop(["Branch Name"], axis=1)
            except:pass

    df = df[~df.index.isin(df[df.apply(lambda row:row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(),axis=1)==True].index)]
    #df = pd.DataFrame(df.T.drop_duplicates().T)
    #df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1, inplace=True)

    df.to_csv("parsed.csv",index=0)
    return df

f = r"C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\ICICI\files\icici011pdf"
#f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\ICICI\files\icici_meena_sarees\icici08.pdf'
try:
    df = tab(f) ; df = process(df)
    print("Parsed")
except:
    print("Not Parsed")

Not Parsed


In [18]:
f = r'C:/Users/MudraCircle/Desktop/bks_raw/Parsing_testing/ICICI/files/icici07.pdf'
tables=camelot.read_pdf(f, flavor="lattice", pages="all")
if len(tables) !=0 :
    df=pd.DataFrame(); tmp=pd.DataFrame();
    for i in range(len(tables)):
        tmp=tables[i].df
        if (tmp.shape[1]==8)|(tmp.shape[1]==7)|(tmp.shape[1]==6)|(tmp.shape[1]==5):
            df=pd.concat([df,tmp]).drop_duplicates(keep="first").reset_index(drop=True)
        else:
            print("\n different structure on page: ",i)
            
df=df.replace(r'^\s*$', np.nan, regex=True)
df=df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)       
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)
try:
    idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
        except:
            try:
                idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('narration').any(), axis=1) ==True].index ][0]
                df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(    drop=True,inplace=True)           
            except:       print("\nProcess-Column headers missing") 
                    
df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
df=pd.DataFrame(df.T.drop_duplicates().T)
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)
     
# try:
#     dft=pd.DataFrame(df.iloc[:,0].dropna().unique().astype(int)).reset_index()
#     if sum(dft.iloc[:,1])-sum(dft.iloc[:,0])==dft.shape[0] :
#         df.drop(df.iloc[:,0].name, axis=1,inplace=True)
# except:  pass
df=df.drop_duplicates()
df.to_csv("parsed.csv", index=0)
print("parsed..")

parsed..


In [39]:
def abc(a):
    if type(a) == str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else:
        z=a
    return z

def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False

def icici_p1(f):
    tables=tabula.read_pdf(f,
                              lattice=True,
                              pages="all",
                              silent=True,
                              multiple_tables=True,
                              pandas_options={'header':None})
    df = pd.DataFrame()
    df = pd.concat([c for c in tables]).drop_duplicates()
    return df

def icici_p1_process(df):
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)

    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('transaction').any(),axis=1)==True].index][0]
            df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
        except:
            try:
                idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('remarks').any(), axis=1) ==True].index ][0]
                df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
            except:
                print("\nICICI Column Headers Missing"); pass
    try:
        idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('page total').any(),axis=1)==True].index][0]    
        df.drop(df.index[idx2:], inplace=True)
    except:pass

    try:
        df = df.loc[:, df.columns.notnull()]
    except:pass

    try:
        dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
        except:pass

    try:
        narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
                except:pass

    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
        df["bal"]=df[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing") 

    try:
        wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
        except: pass 

    try:
        dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
    except:
        try:
            dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
        except: pass

    df[[wdl, dep]] = df[[wdl, dep]].replace({"NA":np.nan, "-":np.nan,"0":np.nan})

    try:
        auto=[c for c in df.columns if "AUTOSWEEP" in str(c).upper() ][0]
        for j,i in enumerate(df[auto]):
            if isnan(i) == False:
                df[wdl][j] = df[auto][j]
            else:
                pass
    except: pass

    try:
        rev=[c for c in df.columns if "REVERSE" in str(c).upper() ][0]
        for j,i in enumerate(df[rev]):
            if isnan(i) == False:
                df[dep][j] = df[rev][j]
            else:
                pass
    except: pass
    
    df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
    df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
    df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)

    try:
        for i in df.index:
        #     # create new column
        #     if "Dr" in df["new_column"][i].astype(str):
        #         df[bal][i] = df[bal][i]*-1
            if df["bal"][i] == "Dr":
                df[bal][i] = df[bal][i]*-1
            else:
                pass
    except:pass

    try:
        df.drop(['bal'], axis=1, inplace=True)
    except:pass

    try:
        df.drop([auto], axis=1, inplace=True)
    except:pass

    try:
        df.drop([rev], axis=1, inplace=True)
    except:pass

    df = df[[dat,chq,narr,wdl,dep,bal]]
    df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]
        
    df.to_csv("check.csv", index=0)

f = r"C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\ICICI\files\meena_icici\icici01.pdf"
try:
    df = icici_p1(f) ; df = icici_p1_process(df)
    print("Parsed")
except Exception as e:
    print("Error:",e)
    print("Not Parsed")

Parsed


In [68]:
def abc(a):
    if type(a) == str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else:
        z=a
    return z

def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False

def icici_p1(f):
    tables=tabula.read_pdf(f,
                              lattice=True,
                              pages="all",
                              silent=True,
                              multiple_tables=True,
                              pandas_options={'header':None})
    df = pd.DataFrame()
    df = pd.concat([c for c in tables]).drop_duplicates()
    return df

def icici_p1_process(df):
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)

    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('transaction').any(),axis=1)==True].index][0]
            df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
        except:
            try:
                idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('remarks').any(), axis=1) ==True].index ][0]
                df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
            except:
                print("\nICICI Column Headers Missing"); pass
    try:
        idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('page total').any(),axis=1)==True].index][0]    
        df.drop(df.index[idx2:], inplace=True)
    except:pass

    try:
        df = df.loc[:, df.columns.notnull()]
    except:pass

    try:
        dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
        except:pass

    try:
        narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
                except:pass

    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
        df["bal"]=df[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing") 

    try:
        wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
        except: pass 

    try:
        dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
    except:
        try:
            dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
        except: pass

    df[[wdl, dep]] = df[[wdl, dep]].replace({"NA":np.nan, "-":np.nan,"0":np.nan})

    try:
        auto=[c for c in df.columns if "AUTOSWEEP" in str(c).upper() ][0]
        for j,i in enumerate(df[auto]):
            if isnan(i) == False:
                df[wdl][j] = df[auto][j]
            else:
                pass
    except: pass

    try:
        rev=[c for c in df.columns if "REVERSE" in str(c).upper() ][0]
        for j,i in enumerate(df[rev]):
            if isnan(i) == False:
                df[dep][j] = df[rev][j]
            else:
                pass
    except: pass
    
    df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
    df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
    df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)

    try:
        for i in df.index:
        #     # create new column
        #     if "Dr" in df["new_column"][i].astype(str):
        #         df[bal][i] = df[bal][i]*-1
            if df["bal"][i] == "Dr":
                df[bal][i] = df[bal][i]*-1
            else:
                pass
    except:pass

    try:
        df.drop(['bal'], axis=1, inplace=True)
    except:pass

    try:
        df.drop([auto], axis=1, inplace=True)
    except:pass

    try:
        df.drop([rev], axis=1, inplace=True)
    except:pass

    df = df[[dat,chq,narr,wdl,dep,bal]]
    df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]
        
    #df.to_csv("check_csv.csv", index=0)
    #df.to_xml("check_xml.xml")
    df.to_excel("icici09.xlsx", index=False)

f = r"C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\ICICI\ICICI\pattern_1\icici09.pdf"
try:
    df = icici_p1(f) ; df = icici_p1_process(df)
    print("Parsed")
except Exception as e:
    print("Error:",e)
    print("Not Parsed")

Parsed


In [ ]:
def to_xml(df, filename="None.xml", mode='w'):
    def row_to_xml(row):
        xml = ['<item>']
        for i, col_name in enumerate(row.index):
            xml.append('  <field name="{0}">{1}</field>'.format(col_name, row.iloc[i]))
        xml.append('</item>')
        return '\n'.join(xml)
    res = '\n'.join(df.apply(row_to_xml, axis=1))

    if filename is None:
        return res
    with open(filename, mode) as f:
        f.write(res)

pd.DataFrame.to_xml = to_xml
